In [ ]:
import pandas as pd

In [ ]:
teams=pd.read_csv("teams.csv")


In [ ]:
teams.head()

In [ ]:
teams=teams[["team","country","year","athletes","age","prev_medals","medals"]]
teams

In [ ]:
teams.corr(numeric_only=True)["medals"]

In [ ]:
import seaborn as sns

In [ ]:
sns.lmplot(x="athletes",y="medals",data=teams,fit_reg=True,ci=None)

In [ ]:
sns.lmplot(x="prev_medals",y="medals",data=teams,fit_reg=True,ci=95)#ci=confidence interval

In [ ]:
sns.lmplot(x="age",y="medals",data=teams,fit_reg=True,ci=None)

In [ ]:
teams.plot.hist(y="medals")

In [ ]:
filtered=teams[(teams["age"]>=18) & (teams["age"]<=27)] 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.scatter(filtered["age"],filtered["medals"],alpha=0.6)
plt.xlabel("Age")
plt.ylabel("Medals")
plt.title("medals vs age(18-27)")
plt.show()

In [ ]:
sns.scatterplot(x="age",y="medals",data=filtered)
plt.title("medals vs age(18-25)")
plt.show()

In [ ]:
sns.lineplot(x="age",y="medals",data=filtered,ci=None)
plt.title("medals vs age(18-25)")
plt.show()

In [ ]:
bins = [18, 20, 22, 24, 26,28,30]  # Bin edges
labels = ["18-19", "20-21", "22-23", "24-25","26-27","28-29"]

teams["age_bin"] = pd.cut(teams["age"], bins=bins, labels=labels)

avg_medals = teams.groupby("age_bin")["medals"].mean().reset_index()

sns.barplot(x="age_bin", y="medals", data=avg_medals)
plt.title("Average Medals by Age Group")
plt.xlabel("age")
plt.ylabel("medals")
plt.show()


In [ ]:
teams=teams.drop("age_bin",axis=1)
teams[teams.isnull().any(axis=1)]

In [ ]:
teams=teams.dropna()
teams

In [ ]:
train=teams[teams["year"]<2012].copy()
test=teams[teams["year"]>=2012].copy()


In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()


In [ ]:
predictors=["athletes","prev_medals"]
target="medals"

In [ ]:
reg.fit(train[predictors],train["medals"])

In [ ]:
predictions=reg.predict(test[predictors])


In [ ]:
test["predictions"]=predictions
test

In [ ]:
test.loc[test["predictions"]<0,"predictions"]=0#convert all negative values to 0
test["predictions"]=test["predictions"].round()
test

In [ ]:
from sklearn.metrics import mean_absolute_error
error=mean_absolute_error(test["medals"],test["predictions"])


In [ ]:
error

In [ ]:
teams.describe()["medals"]#error should be lesser than std

In [ ]:
test[test["team"]=="USA"]

In [ ]:
test[test["team"]=="IND"]

In [ ]:
errors=(test["medals"]-test["predictions"]).abs()

In [ ]:
errors

In [ ]:
error_by_team=errors.groupby(test["team"]).mean()
error_by_team

In [ ]:
medals_by_team=test["medals"].groupby(test["team"]).mean()

In [ ]:
error_ratio=error_by_team/medals_by_team
error_ratio

In [ ]:
error_ratio[~pd.isnull(error_ratio)]#remove nan values(0/0)

In [ ]:
import numpy as np
error_ratio=error_ratio[np.isfinite(error_ratio)]#remove inf values
error_ratio

In [ ]:
error_ratio.plot.hist()

In [ ]:
error_ratio.sort_values()
